In [6]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.Collecting langchain==0.0.284 (from -r requirements.txt (line 1))
  Using cached langchain-0.0.284-py3-none-any.whl.metadata (14 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached streamlit-1.22.0-py2.py3-none-any.whl.metadata (7.3 kB)
  Using cached tiktoken-0.4.0.tar.gz (25 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'



ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement faiss-cpu==1.7.4 (from versions: 1.8.0, 1.8.0.post1, 1.9.0, 1.9.0.post1, 1.10.0, 1.11.0, 1.11.0.post1, 1.12.0, 1.13.0, 1.13.1)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for faiss-cpu==1.7.4


In [1]:
%pip install langchain-huggingface huggingface_hub


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

from dotenv import load_dotenv
import os
load_dotenv("E:\\AI-Buildathon\\.env")
api_key = os.getenv("GOOGLE_GENAI_API_KEY")




In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, api_key=api_key)





In [3]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_pdf_documents(file_path):
    loader = DirectoryLoader(
    file_path,
    glob="**/*.pdf",
    loader_cls=PyPDFLoader,

)
    
    documents=loader.load()
    return documents

document=load_pdf_documents("E:\\AI-Buildathon\\data")


 

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)


In [4]:
for doc in document:
    doc.metadata.update({
        "source": ["WHO","DGHS","UNICEF","The pregnancy encyclopedia","The Gale encyclopedia of medicine"],
        "category": ["disease","symptom","treatment","prevention","diagnosis","Health Tips","Nutrition","Exercise","Mental Health","Medications","Vaccinations"],
        "region": "Bangladesh",
        "language": "en",
        "confidence_level": "primary"
    })


In [5]:
def text_splitter(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=300,
        length_function=len,
    )
    text_chunk = text_splitter.split_documents(document)
    return text_chunk

text_chunk= text_splitter(document)
print(f"Number of Chunks: {len(text_chunk)}") 

Number of Chunks: 8530


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")




In [7]:
load_dotenv("E:\\AI-Buildathon\\.env")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [8]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("test")

In [9]:
from langchain_pinecone import PineconeVectorStore

docstore = PineconeVectorStore.from_documents(
    documents=text_chunk,
    embedding=embedding_model,
    index_name="test",
)


In [10]:
docsearch= PineconeVectorStore.from_existing_index(
    embedding=embedding_model,
    index_name="test"
)

In [15]:
#retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":6})
retriever = docsearch.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 30,
        "fetch_k": 60
    }
)



In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import (create_stuff_documents_chain)

system_prompt=(
    "You are MaBondhu AI, a clinical assistant for maternal health. "
        "Provide safe, evidence-based, detailed guidance. Use the following pieces of retrieved context to answer the question at the end. "
        "Language: {language}.\n\n"
        "Rules:\n"
        "1) If this is potentially life-threatening, say 'EMERGENCY' and recommend immediate referral.\n"
        "2) If unsure, recommend human-in-the-loop evaluation and list needed vitals/tests.\n"
        "3) If the question is about personalized care, nutrition, diet plan, lifestyle, or common symptoms,provide a clear, helpful answer in detail.\n"
        "4) If the question is about giving advices or tips during pregnancy, postpartum, or newborn care, provide a clear, helpful answer in detail.\n"
        "5) If the question is about medications or vaccinations, provide a clear, helpful answer in detail.\n"
        "6) If the question is about antinatal care, labor/delivery, or postpartum care, giving check up schedules, provide a clear, helpful answer in detail.\n"
        "7) ONLY recommend consulting a doctor if the context mentions a danger sign or emergency.\n"
        "8) Include 1–2 specific, actionable next steps (e.g., 'go to nearest clinic', 'take iron supplement 60mg').\n"
        "9) Provide detailed Bangla/English text depending on language.\n\n"
        "10) If you don't know the answer, say 'Sorry, I don't know about it'.\n\n"
        "Be practical, culturally appropriate, and reassuring.\n\n"
        "Answer now in clear {language} with citations (e.g., [WHO Section 3])."
        "\n\n"
        "Context:\n{context}\n\n"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [17]:
question_answering_chain = create_stuff_documents_chain(llm,prompt=prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)



In [18]:

inp=input("Enter your question: ")
response = rag_chain.invoke({
    "input": inp,
    "language": {"English","Bengali" }
})

print(response['answer'])  

Based on the provided context, the World Health Organization (WHO) recommendations on antenatal care (ANC) aim to ensure a "positive pregnancy experience". While the complete "Table 1: Summary list of WHO recommendations on antenatal care (ANC) for a positive pregnancy experience" is referenced as a comprehensive list (Executive summary xi), the full content of this table is not provided in the context.

However, specific recommendations and non-recommendations mentioned include:

**A positive pregnancy experience is defined as:**
*   Maintaining physical and sociocultural normality.
*   Maintaining a healthy pregnancy for mother and baby (including preventing and treating risks, illness and death).
*   Having an effective transition to positive labour and birth.
*   Achieving positive motherhood [WHO page 2].

**Specific ANC recommendations and considerations mentioned:**

1.  **Counselling about healthy eating and keeping physically active during pregnancy** is recommended for pregna